# Azure AI-agenter med støtte for Model Context Protocol (MCP)

Denne notatboken viser hvordan du kan bruke Azure AI-agenter med Model Context Protocol (MCP)-verktøy for å lage en intelligent agent som kan utnytte eksterne MCP-servere for forbedrede funksjoner.


## Installer nødvendige NuGet-pakker

Først må vi installere Azure AI Agents Persistent-pakken, som gir den grunnleggende funksjonaliteten for å arbeide med Azure AI Agents.


## Fordeler med nøkkelfri autentisering

Denne notatboken demonstrerer **nøkkelfri autentisering**, som gir flere fordeler:
- ✅ **Ingen API-nøkler å administrere** - Bruker Azure-basert identitetsautentisering
- ✅ **Forbedret sikkerhet** - Ingen hemmeligheter lagret i kode eller konfigurasjon
- ✅ **Automatisk rotasjon av legitimasjon** - Azure håndterer legitimasjonens livssyklus
- ✅ **Rollebasert tilgang** - Bruker Azure RBAC for detaljerte tillatelser

`DefaultAzureCredential` vil automatisk bruke den beste tilgjengelige kilden til legitimasjon:
1. Managed Identity (når den kjører i Azure)
2. Azure CLI-legitimasjon (under utvikling)
3. Visual Studio-legitimasjon
4. Miljøvariabler (hvis konfigurert)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importer nødvendige navnerom

Importer de nødvendige navnerommene for Azure AI Agents og Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurer Azure AI Agent Client (Nøkkelfri autentisering)

Sett opp konfigurasjonsvariablene og opprett PersistentAgentsClient ved hjelp av **nøkkelfri autentisering**:
- **projectEndpoint**: Endepunktet for Azure AI Foundry-prosjektet
- **modelDeploymentName**: Navnet på den distribuerte AI-modellen (GPT-4.1 nano)
- **mcpServerUrl**: URL-en til MCP-serveren (Microsoft Learn API)
- **mcpServerLabel**: En etikett for å identifisere MCP-serveren
- **DefaultAzureCredential**: Bruker administrert identitet, Azure CLI eller andre legitimasjonskilder (ingen API-nøkler kreves)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Opprett MCP-verktøydefinisjon

Opprett en MCP-verktøydefinisjon som kobler til Microsoft Learn MCP-serveren. Dette vil gjøre det mulig for agenten å få tilgang til Microsoft Learn-innhold og dokumentasjon.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Opprett AI-agenten

Opprett en vedvarende AI-agent med den angitte modellen og MCP-verktøyene. Agenten er konfigurert med:
- GPT-4.1 nano-modellen
- Instruksjoner om å bruke MCP-verktøy for assistanse
- Tilgang til Microsoft Learn MCP-serveren


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Opprett tråd og send melding

Opprett en samtaletråd og send en brukermelding som spør om forskjellen mellom Azure OpenAI og OpenAI. Dette vil teste agentens evne til å bruke MCP-verktøyene for å gi nøyaktig informasjon.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurer MCP-verktøyressurser (Nøkkelfritt)

Sett opp MCP-verktøyressurser. For en virkelig nøkkelfri tilnærming kan du fjerne tilpassede overskrifter hvis MCP-serveren støtter Azure-identitetsbasert autentisering. Eksempelet nedenfor viser hvordan du legger til overskrifter hvis nødvendig, men for nøkkelfrie scenarier kan disse være unødvendige.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Start Agent Run

Opprett og start en kjøring for å behandle brukerens melding. Agenten vil bruke de konfigurerte MCP-verktøyene og ressursene for å generere et svar.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Overvåk kjøring og håndter verktøy-godkjenninger (Keyless)

Overvåk statusen for agentkjøringen og håndter nødvendige verktøy-godkjenninger. Denne løkken:
1. Venter til kjøringen er fullført eller krever handling
2. Godkjenner automatisk MCP-verktøykall når det er nødvendig
3. For keyless autentisering kan det hende at headere ikke er påkrevd hvis MCP-serveren støtter Azure-identitet


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Vise samtaleresultater

Hent og vis alle meldinger i tråden, som viser både brukerens spørsmål og agentens svar. Meldinger vises i kronologisk rekkefølge med tidsstempler og rolleindikatorer.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
